In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
  ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef

In [2]:
data = pd.read_csv(r'..\..\10_fold_cross_validation\train_10folds.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0,0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0,1
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0,1
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0,0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0,3
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0,6
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0,8
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0,5


In [3]:
def run(fold, data):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [4]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    filename = 'LR_' + str(fold_) + '.pkl'
    joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

Fold = 0, AUC = 0.8591065292096219, Accuracy = 0.7992700729927007,           Precision_1 = 0.723404255319149, Precision_0 = 0.8388888888888889          Recall_1 = 0.7010309278350515, Recall_0 = 0.8531073446327684, F1Score = 0.7120418848167538, kappa = 0.5580385946389818, MCC = 0.5582008165095612
Fold = 1, AUC = 0.8756479701788106, Accuracy = 0.8211678832116789,           Precision_1 = 0.7926829268292683, Precision_0 = 0.8333333333333334          Recall_1 = 0.6701030927835051, Recall_0 = 0.903954802259887, F1Score = 0.7262569832402234, kappa = 0.594845796366709, MCC = 0.5994744169452769
Fold = 2, AUC = 0.8626594443473702, Accuracy = 0.7846715328467153,           Precision_1 = 0.7065217391304348, Precision_0 = 0.8241758241758241          Recall_1 = 0.6701030927835051, Recall_0 = 0.847457627118644, F1Score = 0.6878306878306877, kappa = 0.5236607932111497, MCC = 0.5240879820365124


E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

Fold = 3, AUC = 0.8747743025219873, Accuracy = 0.8211678832116789,           Precision_1 = 0.8, Precision_0 = 0.8298969072164949          Recall_1 = 0.6597938144329897, Recall_0 = 0.9096045197740112, F1Score = 0.7231638418079096, kappa = 0.592880101886106, MCC = 0.5988841705131419
Fold = 4, AUC = 0.8800745529733822, Accuracy = 0.8248175182481752,           Precision_1 = 0.7951807228915663, Precision_0 = 0.837696335078534          Recall_1 = 0.6804123711340206, Recall_0 = 0.903954802259887, F1Score = 0.7333333333333334, kappa = 0.6040700824853995, MCC = 0.6081386169878046
Fold = 5, AUC = 0.9068670277826314, Accuracy = 0.8394160583941606,           Precision_1 = 0.7912087912087912, Precision_0 = 0.8633879781420765          Recall_1 = 0.7422680412371134, Recall_0 = 0.8926553672316384, F1Score = 0.7659574468085107, kappa = 0.6439246263807668, MCC = 0.644685048662435


E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

Fold = 6, AUC = 0.8603879084396295, Accuracy = 0.7883211678832117,           Precision_1 = 0.7294117647058823, Precision_0 = 0.8148148148148148          Recall_1 = 0.6391752577319587, Recall_0 = 0.8700564971751412, F1Score = 0.6813186813186812, kappa = 0.5238780034753431, MCC = 0.5264384637888962
Fold = 7, AUC = 0.8469043042693226, Accuracy = 0.7846715328467153,           Precision_1 = 0.6862745098039216, Precision_0 = 0.8430232558139535          Recall_1 = 0.7216494845360825, Recall_0 = 0.8192090395480226, F1Score = 0.7035175879396985, kappa = 0.5346306638263573, MCC = 0.5350469215994972
Fold = 8, AUC = 0.9093515917602997, Accuracy = 0.8357664233576643,           Precision_1 = 0.7524752475247525, Precision_0 = 0.884393063583815          Recall_1 = 0.7916666666666666, Recall_0 = 0.8595505617977528, F1Score = 0.7715736040609136, kappa = 0.6434973688775806, MCC = 0.64400280780206
Fold = 9, AUC = 0.8612876647834276, Accuracy = 0.8021978021978022,           Precision_1 = 0.71, Precision_0 

E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [5]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.799270,0.859107,0.723404,0.838889,0.701031,0.853107,0.712042,0.558039,0.558201
1,0.821168,0.875648,0.792683,0.833333,0.670103,0.903955,0.726257,0.594846,0.599474
2,0.784672,0.862659,0.706522,0.824176,0.670103,0.847458,0.687831,0.523661,0.524088
3,0.821168,0.874774,0.800000,0.829897,0.659794,0.909605,0.723164,0.592880,0.598884
4,0.824818,0.880075,0.795181,0.837696,0.680412,0.903955,0.733333,0.604070,0.608139
5,0.839416,0.906867,0.791209,0.863388,0.742268,0.892655,0.765957,0.643925,0.644685
6,0.788321,0.860388,0.729412,0.814815,0.639175,0.870056,0.681319,0.523878,0.526438
7,0.784672,0.846904,0.686275,0.843023,0.721649,0.819209,0.703518,0.534631,0.535047
8,0.835766,0.909352,0.752475,0.884393,0.791667,0.859551,0.771574,0.643497,0.644003
9,0.802198,0.861288,0.710000,0.855491,0.739583,0.836158,0.724490,0.570304,0.570593


In [6]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.799270,0.859107,0.723404,0.838889,0.701031,0.853107,0.712042,0.558039,0.558201
1,0.821168,0.875648,0.792683,0.833333,0.670103,0.903955,0.726257,0.594846,0.599474
2,0.784672,0.862659,0.706522,0.824176,0.670103,0.847458,0.687831,0.523661,0.524088
3,0.821168,0.874774,0.800000,0.829897,0.659794,0.909605,0.723164,0.592880,0.598884
4,0.824818,0.880075,0.795181,0.837696,0.680412,0.903955,0.733333,0.604070,0.608139
5,0.839416,0.906867,0.791209,0.863388,0.742268,0.892655,0.765957,0.643925,0.644685
6,0.788321,0.860388,0.729412,0.814815,0.639175,0.870056,0.681319,0.523878,0.526438
7,0.784672,0.846904,0.686275,0.843023,0.721649,0.819209,0.703518,0.534631,0.535047
8,0.835766,0.909352,0.752475,0.884393,0.791667,0.859551,0.771574,0.643497,0.644003
9,0.802198,0.861288,0.710000,0.855491,0.739583,0.836158,0.724490,0.570304,0.570593


In [7]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.799270,0.859107,0.723404,0.838889,0.701031,0.853107,0.712042,0.558039,0.558201
1,0.821168,0.875648,0.792683,0.833333,0.670103,0.903955,0.726257,0.594846,0.599474
2,0.784672,0.862659,0.706522,0.824176,0.670103,0.847458,0.687831,0.523661,0.524088
3,0.821168,0.874774,0.800000,0.829897,0.659794,0.909605,0.723164,0.592880,0.598884
4,0.824818,0.880075,0.795181,0.837696,0.680412,0.903955,0.733333,0.604070,0.608139
5,0.839416,0.906867,0.791209,0.863388,0.742268,0.892655,0.765957,0.643925,0.644685
6,0.788321,0.860388,0.729412,0.814815,0.639175,0.870056,0.681319,0.523878,0.526438
7,0.784672,0.846904,0.686275,0.843023,0.721649,0.819209,0.703518,0.534631,0.535047
8,0.835766,0.909352,0.752475,0.884393,0.791667,0.859551,0.771574,0.643497,0.644003
9,0.802198,0.861288,0.710000,0.855491,0.739583,0.836158,0.724490,0.570304,0.570593


In [8]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.799270,0.859107,0.723404,0.838889,0.701031,0.853107,0.712042,0.558039,0.558201
Fold_1,0.821168,0.875648,0.792683,0.833333,0.670103,0.903955,0.726257,0.594846,0.599474
Fold_2,0.784672,0.862659,0.706522,0.824176,0.670103,0.847458,0.687831,0.523661,0.524088
Fold_3,0.821168,0.874774,0.800000,0.829897,0.659794,0.909605,0.723164,0.592880,0.598884
Fold_4,0.824818,0.880075,0.795181,0.837696,0.680412,0.903955,0.733333,0.604070,0.608139
Fold_5,0.839416,0.906867,0.791209,0.863388,0.742268,0.892655,0.765957,0.643925,0.644685
Fold_6,0.788321,0.860388,0.729412,0.814815,0.639175,0.870056,0.681319,0.523878,0.526438
Fold_7,0.784672,0.846904,0.686275,0.843023,0.721649,0.819209,0.703518,0.534631,0.535047
Fold_8,0.835766,0.909352,0.752475,0.884393,0.791667,0.859551,0.771574,0.643497,0.644003
Fold_9,0.802198,0.861288,0.710000,0.855491,0.739583,0.836158,0.724490,0.570304,0.570593


# Final Model

In [9]:
X = data.iloc[:,:-2]
y = data.iloc[:,-2]

In [10]:
model = LogisticRegression()
model.fit(X, y)

E:\MY_FILES\Anaconda\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [11]:
joblib.dump(model, 'LR_Final.pkl')

['LR_Final.pkl']